In [17]:
import cv2

def GetEyes(frame):
    # Umwandeln in Graustufenbild
    grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Haar-Cascade-Datei für Augen laden
    eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
    # Erkennen der Augen
    eyes = eye_cascade.detectMultiScale(grayFrame, scaleFactor=1.1, minNeighbors=4)
    #print("Augen: ",eyes)
    return eyes
  
vid = cv2.VideoCapture(0)
  
while(True):
      
    # Capture the video frame
    # by frame
    ret, frame = vid.read()

    #Augen Position erfassen
    eyes = GetEyes(frame)

    for (x, y, w, h) in eyes:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
  
    # Anzeigen des Frames
    cv2.imshow('Video', frame)
      
    #Beenden mit der Taste q
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x1cd7070) is not the object's thread (0x2716af0).
Cannot move to target thread (0x1cd7070)

QObject::moveToThread: Current thread (0x1cd7070) is not the object's thread (0x2716af0).
Cannot move to target thread (0x1cd7070)

QObject::moveToThread: Current thread (0x1cd7070) is not the object's thread (0x2716af0).
Cannot move to target thread (0x1cd7070)

QObject::moveToThread: Current thread (0x1cd7070) is not the object's thread (0x2716af0).
Cannot move to target thread (0x1cd7070)

QObject::moveToThread: Current thread (0x1cd7070) is not the object's thread (0x2716af0).
Cannot move to target thread (0x1cd7070)

QObject::moveToThread: Current thread (0x1cd7070) is not the object's thread (0x2716af0).
Cannot move to target thread (0x1cd7070)

QObject::moveToThread: Current thread (0x1cd7070) is not the object's thread (0x2716af0).
Cannot move to target thread (0x1cd7070)

QObject::moveToThread: Current thread (0x1cd7070) is not the object's thread